### setup:
put experiment tag in the 13th column of the mouse colony spreadsheet
download the tab as a .csv

In [13]:
import numpy as np
import pandas as pd
import os

In [93]:
exp_ID = 'RAM' #tag in the spreadsheet identifying subjects
filename = 'Mouse Colony log-2020-9 - Syt7.csv'
colony_path = 'Colony' #path to folder containing colony csv's

In [102]:
colony = pd.read_csv(os.path.join(colony_path, filename), header=1, usecols=(2,4,5,6,7,8,9,13))
colony.head(3)

,Cage Tag,Mouseline,ID,Sex,Ear notch,Genotype,DOB,Unnamed: 13
0,51CM,Syt8,51C,M4,RnRn,+/-,09/10/2021,Injected with Dlight 12/21/2021 in striatum on...
1,53BM.2,Syt7,53B,M9,RnRn,-/-,12/19/2021,Injected with DLight 2/1/2022 in midbrain bila...
2,54AF,Syt7,54A,F1,Rn,-/-,12/3/2021,Injected with DLight 1/17/2022 in striatum on ...


RAM cages: 54BM, 57AM, 53CM

### current issue:
fewer cage IDs than cages; must have been a big litter and they got split up into multiple cages

might have to write something fancy (annoying) using the cage counter

In [97]:
#find subjects (rows of colony dataframe) with experimental tag in column 13
subjects = colony.loc[(~pd.isna(colony['Unnamed: 13']))&(colony['Unnamed: 13'].str.contains(exp_ID))]# = cage_ID.ID[i]
cages = subjects['ID'].unique() #note, not sure if cage tag or ID is a better indicator
print(f"{len(cages)} cages found, {len(subjects)} subjects total")

3 cages found, 12 subjects total


In [10]:
os.listdir(colony_path)

['Mouse Colony log-2020-9 - Syt3.csv', 'Mouse Colony log-2020-9 - Syt7.csv']

In [9]:
colony_path = 'Colony'

filenames = os.listdir(colony_path):


Mouse Colony log-2020-9 - Syt3.csv
Mouse Colony log-2020-9 - Syt7.csv


In [ ]:
#import mouse colony data
S3col = pd.read_csv('Colony/actual_database/Syt3.csv', header=1, usecols=(2,4,5,6,7,8,9))
S7col = pd.read_csv('Colony/actual_database/Syt7.csv', header=1, usecols=(2,4,5,6,7,8,9))
colonydata = pd.concat([S3col, S7col], ignore_index=True).dropna()

#import blinded cage IDs
cage_ID = pd.read_csv('Colony/cage_IDs.csv',index_col=None)

print(len(colonydata), 'animals in colony data')

In [ ]:
os.chdir('Syt7_WM/RAM/') #change current directory
groom_path = 'Grooming/groom_times' #path to csv's of grooming durations

filenames_list= []
d = {}

for filename in os.listdir(groom_path):
    if filename.endswith('.csv'):
        f = filename.split(".")[0]
        ID = f[10:]  #split after YYMMDD_CL
        df = pd.read_csv(os.path.join(groom_path, filename), header=None, usecols=(1,2), names=['duration','timestamp'])
        
        for i in range(len(df)): #convert timestamps to seconds
            D = datetime.strptime(df.duration[i], '%H:%M:%S.%f')
            deltaD = timedelta(minutes=D.minute, seconds=D.second, microseconds=D.microsecond)
            df.duration[i] = deltaD.total_seconds()
            T = datetime.strptime(df.timestamp[i], '%H:%M:%S.%f')
            deltaT = timedelta(minutes=T.minute, seconds=T.second, microseconds=T.microsecond)
            df.timestamp[i] = deltaT.total_seconds()

        d[ID] = {'Grooming': df}

print(len(d), 'files found')

In [ ]:
#extract experimental subject data from colony data
cagetags = [x[5:].replace('_','') for x in cage_ID.Cage.tolist()]
allcages = '|'.join(cagetags)
subjectdata = colonydata.loc[colonydata['Cage Tag'].str.contains(allcages)]
subjectdata['Sex'] = [x[0] for x in subjectdata['Sex']] #remove number from sex label

#add blinded cage IDs (Cage1, Cage2, etc) to experimental subject dataframe
for i in range(len(cage_ID)):
    [line, cage, sex] = cage_ID.Cage[i].split('_')
    subjectdata.loc[(subjectdata['Mouseline']==line)&(subjectdata['Cage Tag']==cage+sex), 'Cage ID'] = cage_ID.ID[i]
    
print(len(subjectdata), 'subjects extracted')
if len(d) > len(subjectdata):
    print('(subject data missing...)')